In [1]:
# Install these if not there
# !pip install pydub
# !pip install light-the-torch
# !pip install  git+https://github.com/hmmlearn/hmmlearn.git
# !pip install  git+https://github.com/pyannote/pyannote-audio.git@develop

In [2]:
from pyannote.audio import Pipeline
import whisper
import torch
from datetime import timedelta
from pydub import AudioSegment
from pathlib import Path
import re
import json
import os

In [3]:
device = torch.device("cuda")

In [4]:
device

device(type='cuda')

### With first usage of whisper first needs to install/update "numba.jit" - happens automatically - 1.42gb

# Generate a token from hugginface and insert it below
1. visit hf.co/pyannote/speaker-diarization and accept user conditions
2. visit hf.co/pyannote/segmentation and accept user conditions
3. visit hf.co/settings/tokens to create an access token
4. instantiate pretrained speaker diarization pipeline

In [5]:
hf_token = "TOKEN"

# convert m4a to wav and create subfolder structure

In [6]:
data = "./dataFlo" # Folder with all your m4a audio files nright next to this file
dataOut = "./outFlo"
SPEAKER_AMOUNT = 3
types = ('**/*.m4a', '**/*.mp3', '**/*.ogg', '**/*.aac')
audio_files = []
for fileending in types:
    pathlist = Path(data).glob(fileending)
    for audio_file in pathlist:
        f = f"{str(audio_file).split('.')[0]}"
        print(f.split("/")[1])
        # Create folders
        !mkdir {f} 
        ftmp = f"{f}/tmp"
        !mkdir {ftmp}
        fa = f"{f}/{str(audio_file).split('/')[1].split('.')[0]}.wav"
        tmp = {"name": f.split("/")[1], "og_audio": fa, "tmp_folder": ftmp}
        audio_files.append(tmp)
        # !ffmpeg -y -i {audio_file} -ss 00:00:30 -to 00:02:30 {fa} # create files which are only 2min long to test
        !ffmpeg -y -i {audio_file} {fa}

Florian_Chemnitz_V2_Handy
mkdir: cannot create directory ‘dataFlo/Florian_Chemnitz_V2_Handy’: File exists
mkdir: cannot create directory ‘dataFlo/Florian_Chemnitz_V2_Handy/tmp’: File exists
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libsh

In [7]:
audio_files

[{'name': 'Florian_Chemnitz_V2_Handy',
  'og_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V2_Handy/tmp'},
 {'name': 'Florian_Chemnitz_V1_PC',
  'og_audio': 'dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V1_PC/tmp'}]

# Append time before actual audio, else pyannot might not pick up the first 30 seconds

In [8]:
spacermilli = 2000
for file in audio_files:
    processed_audio_file = file["og_audio"]
    processed_audio_file2 = file["og_audio"].split(".")[0] +  "_prep.wav"
    file["proc_audio"] = processed_audio_file2
    spacer = AudioSegment.silent(duration=spacermilli)
    audio = AudioSegment.from_wav(processed_audio_file) 
    audio = spacer.append(audio, crossfade=0)
    audio.export(processed_audio_file2, format='wav')
audio_files

[{'name': 'Florian_Chemnitz_V2_Handy',
  'og_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V2_Handy/tmp',
  'proc_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy_prep.wav'},
 {'name': 'Florian_Chemnitz_V1_PC',
  'og_audio': 'dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V1_PC/tmp',
  'proc_audio': 'dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC_prep.wav'}]

# Init Pyannot pipeline for speaker identification and whisper for transcription

In [9]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=hf_token)
device = torch.device("cuda")
model = whisper.load_model('large', device = device)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


# Create diarization file (when does a new speaker speak) - this will take a long while

In [10]:
def create_diarization_file(audiofile, numspeaker):
    if numspeaker == None:
        dz = pipeline(audiofile)  
    else:
        dz = pipeline(audiofile, num_speakers=numspeaker) 
    diarization_file = audiofile.split(".")[0] + "_dia.txt"
    print(diarization_file)
    if os.path.isfile(diarization_file):
        return diarization_file
    with open(diarization_file, "w") as text_file:
        text_file.write(str(dz))
    return diarization_file

def millisec(timeStr):
    spl = timeStr.split(":")
    s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
    return s

In [11]:
for file in audio_files:
    file["dia_file"] = create_diarization_file(file["proc_audio"], SPEAKER_AMOUNT)
audio_files

dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy_prep_dia.txt
dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC_prep_dia.txt


[{'name': 'Florian_Chemnitz_V2_Handy',
  'og_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V2_Handy/tmp',
  'proc_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy_prep.wav',
  'dia_file': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy_prep_dia.txt'},
 {'name': 'Florian_Chemnitz_V1_PC',
  'og_audio': 'dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V1_PC/tmp',
  'proc_audio': 'dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC_prep.wav',
  'dia_file': 'dataFlo/Florian_Chemnitz_V1_PC/Florian_Chemnitz_V1_PC_prep_dia.txt'}]

# Create audio segments based on the previous timestamps

In [12]:
print("test")

test


In [13]:
def create_segments(file):
    dzs = open(file["dia_file"]).read().splitlines()
    groups = []
    g = []
    lastend = 0
    for d in dzs:   
        if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
            groups.append(g)
            g = []

        g.append(d)

        end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
        end = millisec(end)
        if (lastend > end):       #segment engulfed by a previous segment
            groups.append(g)
            g = [] 
        else:
            lastend = end
    if g:
        groups.append(g)
    audio = AudioSegment.from_wav(file["proc_audio"])
    
    gidx = -1
    tmp_files = []
    for g in groups:
        start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
        end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
        start = millisec(start) #- spacermilli
        end = millisec(end)  #- spacermilli
        gidx += 1
        fname = f"{file['tmp_folder']}/{gidx}.wav"
        tmp_files.append(fname)
        audio[start:end].export(fname, format='wav')
    print(tmp_files[:10])
    return tmp_files, groups 

In [14]:
for file in audio_files:
    file["tmp_wavs"], file["groups"] = create_segments(file)
audio_files

['dataFlo/Florian_Chemnitz_V2_Handy/tmp/0.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/1.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/2.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/3.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/4.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/5.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/6.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/7.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/8.wav', 'dataFlo/Florian_Chemnitz_V2_Handy/tmp/9.wav']
['dataFlo/Florian_Chemnitz_V1_PC/tmp/0.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/1.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/2.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/3.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/4.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/5.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/6.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/7.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/8.wav', 'dataFlo/Florian_Chemnitz_V1_PC/tmp/9.wav']


[{'name': 'Florian_Chemnitz_V2_Handy',
  'og_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy.wav',
  'tmp_folder': 'dataFlo/Florian_Chemnitz_V2_Handy/tmp',
  'proc_audio': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy_prep.wav',
  'dia_file': 'dataFlo/Florian_Chemnitz_V2_Handy/Florian_Chemnitz_V2_Handy_prep_dia.txt',
  'tmp_wavs': ['dataFlo/Florian_Chemnitz_V2_Handy/tmp/0.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/1.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/2.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/3.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/4.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/5.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/6.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/7.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/8.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/9.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/10.wav',
   'dataFlo/Florian_Chemnitz_V2_Handy/tmp/11.wav',
   'dataFlo/Florian_Chemnitz_V2

# Create Transcripts for each segment

In [15]:
def create_segment_transcripts(file, language="en"):
    # print(len(file["tmp_wavs"]))
    for i in range(len(file["tmp_wavs"])):
        audiof = str(i) + '.wav'
        segment_audio = f"{file['tmp_folder']}/{audiof}"
        result = model.transcribe(audio=segment_audio, language=language, word_timestamps=True)#, initial_prompt=result.get('text', ""))
        o = f"{file['tmp_folder']}/"
        tmp_files = []
        # print(f"{o}{i}.json")
        with open(f"{o}{i}.json", "w") as outfile:
            tmp_files.append(str(outfile))
            json.dump(result, outfile, indent=4) 
    return tmp_files

In [16]:
for file in audio_files:
    file["tmp_jsons"] = create_segment_transcripts(file, language="de")

# Create txt file with all spoken text split by speaker

You can edit names of the speakers below

In [17]:
speakers = {
    'SPEAKER_00': ('SPEAKER_00', '#e1ffc7', 'darkgreen'),
    'SPEAKER_01': ('SPEAKER_01', 'white', 'darkorange'),
    'SPEAKER_02': ('SPEAKER_02', 'lightblue', 'darkblue'),
    'SPEAKER_03': ('SPEAKER_03', 'lightpink', 'darkred'),
    'SPEAKER_04': ('SPEAKER_04', '#f0e68c', 'goldenrod'),
    'SPEAKER_05': ('SPEAKER_05', 'lightcoral', 'darkred'),
    'SPEAKER_06': ('SPEAKER_06', '#98fb98', 'forestgreen'),
    'SPEAKER_07': ('SPEAKER_07', '#dda0dd', 'purple'),
    'SPEAKER_08': ('SPEAKER_08', '#ffcccb', 'darkred'),
    'SPEAKER_09': ('SPEAKER_09', 'lightcyan', 'darkcyan')
}
def_boxclr = 'white'
def_spkrclr = 'orange'

In [18]:
def timeStr(t):
    return '{0:02d}:{1:02d}:{2:06.2f}'.format(round(t // 3600), round(t % 3600 // 60), t % 60)
def create_capfile(file):
    txt = list("")
    gidx = -1
    last_speaker = ""
    for g in file["groups"]:  
        shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
        shift = millisec(shift) - spacermilli #the start time in the original video
        shift=max(shift, 0)

        gidx += 1
        o = f"{file['tmp_folder']}/"
        captions = json.load(open(f"{o}{gidx}.json"))['segments']

        if captions:
            speaker = g[0].split()[-1]
            boxclr = def_boxclr
            spkrclr = def_spkrclr
        if speaker in speakers:
            speaker, boxclr, spkrclr = speakers[speaker] 

        tmp_txt = ""
        tmp_timestamp = ""
        
        for c in captions:
            start = shift + c['start'] * 1000.0 
            start = start / 1000.0   #time resolution ot youtube is Second.            
            end = (shift + c['end'] * 1000.0) / 1000.0   
            if speaker != last_speaker:
                # print(speaker)
                tmp_txt = ""
                tmp_timestamp = f'[{timeStr(start)} --> {timeStr(end)}] [{speaker}]'
            tmp_txt += f'{c["text"]}'

            last_speaker = speaker
        txt.append(f'{tmp_timestamp}{tmp_txt}\n')


    capfile = f"{dataOut}/{file['proc_audio'].split('.')[0].split('/')[2]}_capspeaker.txt"

    with open(capfile, "w", encoding='utf-8') as file:
        s = "".join(txt)
        file.write(s)
        print(f'captions saved to {capfile}:')

# Create the actual transcribed files with speaker differentiation

In [19]:
for file in audio_files:
    create_capfile(file)

captions saved to ./outFlo/Florian_Chemnitz_V2_Handy_prep_capspeaker.txt:
captions saved to ./outFlo/Florian_Chemnitz_V1_PC_prep_capspeaker.txt:


In [20]:
print("done")

done
